In [258]:
import pandas as pd
import numpy as np
import math
import re

# Matching data cleaning
### Removing duplicates

In [259]:
matching_data = pd.read_excel("matching_data/nav_vs_pronto_item.xlsx", header=1)
matching_data = matching_data.rename(columns={"Code":"nav_code", "Item Description": "nav_item_description", "UOM": "nav_uom", \
                              "Code.1": "pronto_code", "Item Description.1": "pronto_item_description", "UOM.1": "pronto_uom"})
matching_data

,nav_code,nav_item_description,nav_uom,Unnamed: 3,pronto_code,pronto_item_description,pronto_uom
0,28009,Mega Bin Double Breast (F) S/On #20-24 500kg R...,KG,NaN,30001,CHICKEN DOUBLE BREAST (F) S/ON #20-24 500KG RS...,KG
1,27039,Mega Bin Double Breast (F) Skin On #20-24 500kg,KG,NaN,30842,CHICKEN DOUBLE BREAST (F) S/ON #20-24 500KG ME...,KG
2,29124,Chicken 1/2 Breast (F) Skin On 1kg bag,KG,NaN,31152,CHICKEN 1/2 BREAST (F) S/ON RW,KG
3,28991,Chicken 1/2 Breast (F) Skin On 2kg Vac,KG,NaN,31349,CHICKEN 1/2 BREAST (F) S/ON 2KG VAC,KG
4,27932,Chicken 1/2 Breast (F) Skin On RW 15kg Bulk,KG,NaN,10253,CHICKEN 1/2 BREAST (F) S/ON 15KG BULK,KG
...,...,...,...,...,...,...,...
1903,28552,Slice - Hedgehog 10 x 100g (15) Chef Momo's,PKT,NaN,30836,FZ SLICE-HEDGEHOG 10 X 100G(15) CHEF MOMO'S,PK
1904,28555,Slice - GF Nutella 10x 100g (15) Chef Momo's,PKT,NaN,30835,FZ SLICE-NUTELLA GLUTEN FREE 10X 100G(15) CHEF...,PK
1905,28551,Slice - Carrot Cake 10 x 100g (15) Chef Momo's,PKT,NaN,30833,FZ SLICES-CARROT CAKE 10 X 100G(15) CHEF MOMO'S,PK
1906,28556,Slice - Mixed Box 18 x 100g Chef Momo's,CTN,NaN,30832,FZ SLICES-MIXED BOX 18 X 100G CHEF MOMO'S,CTN


In [260]:
nav_system_code = matching_data["nav_code"]
pronto_system_code = matching_data["pronto_code"]

In [261]:
print("Nav system code original data sets: ", nav_system_code.count(), "data sets after removed duplicates: ", len(nav_system_code.unique()))
print("Pronto system code original data sets: ", pronto_system_code.count(), "data sets after removed duplicates: ", len(pronto_system_code.unique()))

Nav system code original data sets:  1908 data sets after removed duplicates:  1905
Pronto system code original data sets:  1908 data sets after removed duplicates:  1908


In [262]:
matching_only_code = matching_data[["nav_code", "pronto_code"]]
print("Duplicate matching code", matching_only_code[matching_only_code.duplicated(["nav_code"], keep=False)], sep='\n')

Duplicate matching code
     nav_code  pronto_code
77      10037        10037
78      10037        31334
402     27274        31353
403     27274        31354
433     27714        70515
438     27714        17518


In [263]:
duplicated_data_sets = matching_data[matching_data[["nav_code", "pronto_code"]].duplicated(["nav_code"], keep=False)]
print("Duplicate Matching rows based on code column are:", duplicated_data_sets, sep='\n')
duplicated_data_sets

Duplicate Matching rows based on code column are:
     nav_code                     nav_item_description nav_uom  Unnamed: 3  \
77      10037  Chicken Butterfly Breast 200g 15kg Bulk      KG         NaN   
78      10037  Chicken Butterfly Breast 200g 15kg Bulk      KG         NaN   
402     27274         Chicken Diced Breast 3x3 5kg Bag      KG         NaN   
403     27274         Chicken Diced Breast 3x3 5kg Bag      KG         NaN   
433     27714         Chicken Diced Thigh (F) 2x2 15kg      KG         NaN   
438     27714         Chicken Diced Thigh (F) 2x2 15kg      KG         NaN   

     pronto_code                        pronto_item_description pronto_uom  
77         10037   CHICKEN BUTTERFLY BREAST (F) 200G 15KG BULK          KG  
78         31334  CHICKEN BUTTERFLY BREAST (F) 200G 5KG BAG RW          KG  
402        31353      CHICKEN BREAST (F) DICE 3X3CM 5KG BAG RW          KG  
403        31354       CHICKEN BREAST (F) DICE 3X3CM 15KG BULK          KG  
433        70515  

,nav_code,nav_item_description,nav_uom,Unnamed: 3,pronto_code,pronto_item_description,pronto_uom
77,10037,Chicken Butterfly Breast 200g 15kg Bulk,KG,NaN,10037,CHICKEN BUTTERFLY BREAST (F) 200G 15KG BULK,KG
78,10037,Chicken Butterfly Breast 200g 15kg Bulk,KG,NaN,31334,CHICKEN BUTTERFLY BREAST (F) 200G 5KG BAG RW,KG
402,27274,Chicken Diced Breast 3x3 5kg Bag,KG,NaN,31353,CHICKEN BREAST (F) DICE 3X3CM 5KG BAG RW,KG
403,27274,Chicken Diced Breast 3x3 5kg Bag,KG,NaN,31354,CHICKEN BREAST (F) DICE 3X3CM 15KG BULK,KG
433,27714,Chicken Diced Thigh (F) 2x2 15kg,KG,NaN,70515,CHICKEN THIGH (F) DICE 2X2CM 5KG BAG,KG
438,27714,Chicken Diced Thigh (F) 2x2 15kg,KG,NaN,17518,CHICKEN THIGH (F) DICE 2X2CM 15KG BULK,KG


In [264]:
def find_duplicated():
    
    duplicated_list = []
    
    nav_des = duplicated_data_sets["nav_item_description"].str.lower()
    pronto_des = duplicated_data_sets["pronto_item_description"].str.lower()
    
    for index,item in nav_des.items():
        nav_kg = re.search(r"\w+kg", item).group()
        pronto_kg = re.search(r"\w+kg", pronto_des.at[index]).group()
        
        if pronto_kg != nav_kg:
            duplicated_list.append(index)
    
    return duplicated_list

duplicated_lists = find_duplicated()
duplicated_lists

[78, 403, 433]

In [265]:
cleaned_matching_data = matching_only_code.drop([78, 403, 433], axis=0)
cleaned_matching_data

,nav_code,pronto_code
0,28009,30001
1,27039,30842
2,29124,31152
3,28991,31349
4,27932,10253
...,...,...
1903,28552,30836
1904,28555,30835
1905,28551,30833
1906,28556,30832


# Input data cleaning
### Find potential issue

In [9]:
data = pd.read_excel("input_data/weekly_report_by_item.xlsx")
data

,Weekly Report By Item,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,04/05/2020 - 09/05/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Closing Inventory Detail,NaN,NaN,NaN,NaN,NaN
2,Category,Sub-Category,Code,Item Description,Refrigeration Code,Inventory Posting Group,New Discontinued Item,ERP Stock,UOM,Cost Price,Last Cost,Inventory Value,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fz Chicken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3404,Fz Chicken,NaN,29804,Chicken Maryland Skin Off XL 15Kg Bulk Frozen,FROZEN,PRODUCED,NaN,300,KG,3,NaN,900,NaN
3405,Fz Chicken,NaN,29775,Retail Silkie Chicken Whole R/W #5 500-599g (25),FROZEN,PURCHASED,NaN,49,PCS,14.3,NaN,700.7,NaN
3406,Fz Chicken,29512,29517,Retail Frozen Cooked Drumsticks Sec 1kg,FROZEN,PURCHASED,NaN,14,PACK,3.5,NaN,49,NaN
3407,Fz Chicken,NaN,29512,Frozen Cooked Drumsticks Sec 1kg (10),FROZEN,PURCHASED,NaN,13,CTN,35,NaN,455,NaN


In [10]:
input_data = pd.read_excel("input_data/weekly_report_by_item.xlsx", header=3)
input_data = input_data.loc[:, ~input_data.columns.str.contains("^Unnamed")]
input_data.head()

,Category,Sub-Category,Code,Item Description,Refrigeration Code,Inventory Posting Group,New Discontinued Item,ERP Stock,UOM,Cost Price,Last Cost,Inventory Value
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fz Chicken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fz Chicken,NaN,10684.0,"Boiler Chicken 12"" 9pc/ctn - Frozen",FROZEN,PURCHASED,NaN,865.0,PCS,2.88,2.86,2491.2
4,Fz Chicken,NaN,29220.0,"Retail Boiler Chicken 12"" - Frozen",FROZEN,PURCHASED,NaN,98.0,PCS,27.50,27.50,2695.0


In [11]:
input_data.shape

(3406, 12)

In [12]:
code_not_nan_data_sets = input_data[input_data["Code"].notna()]
code_not_nan_data_sets.shape

(3310, 12)

# Merging data

In [13]:
merged_data = pd.merge(code_not_nan_data_sets, cleaned_matching_data, \
                      how="left", left_on="Code", right_on="nav_code")
merged_data.head()

,Category,Sub-Category,Code,Item Description,Refrigeration Code,Inventory Posting Group,New Discontinued Item,ERP Stock,UOM,Cost Price,Last Cost,Inventory Value,nav_code,pronto_code
0,Fz Chicken,NaN,10684.0,"Boiler Chicken 12"" 9pc/ctn - Frozen",FROZEN,PURCHASED,NaN,865.0,PCS,2.88,2.86,2491.2,10684.0,10684.0
1,Fz Chicken,NaN,29220.0,"Retail Boiler Chicken 12"" - Frozen",FROZEN,PURCHASED,NaN,98.0,PCS,27.50,27.50,2695.0,NaN,NaN
2,Fz Chicken,NaN,28826.0,Diced Boiler Chicken 2X2cm,FRESH,PRODUCED,NaN,0.0,PCS,3.88,NaN,0.0,28826.0,30893.0
3,Fz Chicken,NaN,28766.0,Dog Bone RW,FROZEN,PRODUCED,NaN,81.4,CTN,1.00,NaN,81.4,28766.0,31107.0
4,Fz Chicken,NaN,28543.0,Chicken - Cooked Karaage 2.5kg (2) Melco Frozen,FROZEN,PURCHASED,NaN,147.0,PKT,27.00,27.00,3969.0,28543.0,28379.0


## merging issue

In [14]:
data_without_matching = merged_data[merged_data["nav_code"].isna()]
data_without_matching

,Category,Sub-Category,Code,Item Description,Refrigeration Code,Inventory Posting Group,New Discontinued Item,ERP Stock,UOM,Cost Price,Last Cost,Inventory Value,nav_code,pronto_code
1,Fz Chicken,NaN,29220.0,"Retail Boiler Chicken 12"" - Frozen",FROZEN,PURCHASED,NaN,98.0,PCS,27.5,27.5,2695.0,NaN,NaN
5,Fz Chicken,NaN,29429.0,Retail Chicken - Cooked Karaage 2.5kg (2) Melco,FROZEN,PURCHASED,NaN,6.0,PK,27.0,NaN,162.0,NaN,NaN
8,Fz Chicken,NaN,29749.0,Retail Salt-Baked Chicken 1pcs (10),FROZEN,PURCHASED,NaN,16.0,PCS,13.0,NaN,208.0,NaN,NaN
9,Fz Chicken,NaN,29716.0,Cooked Whole Chicken (3) - Melco,FROZEN,PURCHASED,NaN,0.0,CTN,35.4,NaN,0.0,NaN,NaN
13,Fz Chicken,NaN,29113.0,Chicken 1/2 Breast (F) 10kg Bulk - Frozen,FROZEN,PRODUCED,NaN,8530.0,KG,5.0,NaN,42650.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3295,Vegetable,NaN,29573.0,Retail Veg -Premium Pre-packed Choy Sum 1 bunc...,FRESH,PURCHASED,NaN,0.0,PK,1.2,NaN,0.0,NaN,NaN
3296,Vegetable,NaN,29575.0,Retail Veg- Tong Ho 1bunch/pk,FRESH,PURCHASED,NaN,0.0,PK,NaN,NaN,0.0,NaN,NaN
3300,Vegetable,NaN,29734.0,Retail Premium Truss Tomato 1kg/pk,FRESH,PURCHASED,NaN,1.0,PK,NaN,NaN,0.0,NaN,NaN
3306,Fz Chicken,NaN,29775.0,Retail Silkie Chicken Whole R/W #5 500-599g (25),FROZEN,PURCHASED,NaN,49.0,PCS,14.3,NaN,700.7,NaN,NaN


## compose data

In [15]:
merge_with_original_data = pd.merge(input_data, cleaned_matching_data, \
                                   how="left", left_on="Code", right_on="nav_code")
merge_with_original_data.pop("nav_code")
merge_with_original_data.insert(3, "pronto_code", merge_with_original_data.pop("pronto_code"))
merge_with_original_data

,Category,Sub-Category,Code,pronto_code,Item Description,Refrigeration Code,Inventory Posting Group,New Discontinued Item,ERP Stock,UOM,Cost Price,Last Cost,Inventory Value
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fz Chicken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fz Chicken,NaN,10684.0,10684.0,"Boiler Chicken 12"" 9pc/ctn - Frozen",FROZEN,PURCHASED,NaN,865.0,PCS,2.88,2.86,2491.2
4,Fz Chicken,NaN,29220.0,NaN,"Retail Boiler Chicken 12"" - Frozen",FROZEN,PURCHASED,NaN,98.0,PCS,27.50,27.50,2695.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3401,Fz Chicken,NaN,29804.0,31317.0,Chicken Maryland Skin Off XL 15Kg Bulk Frozen,FROZEN,PRODUCED,NaN,300.0,KG,3.00,NaN,900.0
3402,Fz Chicken,NaN,29775.0,NaN,Retail Silkie Chicken Whole R/W #5 500-599g (25),FROZEN,PURCHASED,NaN,49.0,PCS,14.30,NaN,700.7
3403,Fz Chicken,29512,29517.0,NaN,Retail Frozen Cooked Drumsticks Sec 1kg,FROZEN,PURCHASED,NaN,14.0,PACK,3.50,NaN,49.0
3404,Fz Chicken,NaN,29512.0,27057.0,Frozen Cooked Drumsticks Sec 1kg (10),FROZEN,PURCHASED,NaN,13.0,CTN,35.00,NaN,455.0


In [16]:
merge_with_original_data.to_excel("output_data/test.xlsx", index_label=False)

In [17]:
a = merge_with_original_data[["Category", "Code"]]
category_list = list(a[a["Category"].notna() & a["Code"].isna()].index)

## Excel formatting

In [243]:
writer = pd.ExcelWriter("output_data/test_writer.xlsx", engine="xlsxwriter")

merge_with_original_data.style.set_caption("Weekly Report")
merge_with_original_data.to_excel(writer, sheet_name="Weekly Report", index=False, header=False, startrow=4)

In [244]:
workbook = writer.book
worksheet = writer.sheets["Weekly Report"]
total_rows = merge_with_original_data.shape[0] + 4
total_cols = merge_with_original_data.shape[1]

In [245]:
# Format variable
font_size = 12
normal_column_width = 10

In [246]:
# Add title and header
title_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'vcenter',
    'align': 'center',
    'fg_color': '#BFBFBF',
    'border': 1,
    'font_size': 20
})

def weekly_report_title(title):
    titles = title.split(", ")
    worksheet.write('A2', titles[1])
    
    worksheet.merge_range('A1:D1', titles[0], title_format)    
    worksheet.merge_range("I3:M3", "Closing Inventory Detail", title_format)

weekly_report_title("Weekly Report By Item, 04/05/2020 - 09/05/2020")

In [247]:
# Add header
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'vcenter',
    'align': 'center',
    'fg_color': '#BFBFBF',
    'border': 1,
    'font_size': font_size
})

for col_num, value in enumerate(merge_with_original_data.columns.values):
    worksheet.write(3, col_num, value, header_format)

In [248]:
# Categroy format
categroy_format = workbook.add_format({
    'bold': True,
    'italic': True,
    'font_size': 18,
    'top': 1,
    'bottom': 1
})

for index in category_list:
    row = index + 5
    category_title = merge_with_original_data.iloc[index, 0]
    worksheet.merge_range('A{}:M{}'.format(row, row), category_title, categroy_format)


In [249]:
merge_with_original_data.iloc[1, 0]

'Fz Chicken'

In [250]:
# General column format for A:B
a_b_cell_format = workbook.add_format({
    'font_size': font_size
})

worksheet.set_column('A:B', 10, a_b_cell_format)

0

In [251]:
# Set column format for C and D
c_d_cell_format = workbook.add_format({
    'font_size': font_size,
    'align': 'center'
})

worksheet.set_column("C:D", normal_column_width, c_d_cell_format)

c_cell_format = workbook.add_format({
    'bg_color': '#9BBB59'
})

worksheet.conditional_format('C5:C{}'.format(total_rows), {
    'type': 'no_blanks',
    'format': c_cell_format
})


d_cell_format = workbook.add_format({
    'bg_color': '#F79646'
})

worksheet.conditional_format('D5:D{}'.format(total_rows), {
    'type': 'no_blanks',
    'format': d_cell_format
})

In [252]:
# Item description column format for E
e_cell_format = workbook.add_format({
    'font_size': font_size
})

worksheet.set_column('E:E', 50, e_cell_format)

0

In [253]:
# F to H column format
f_h_cell_format = workbook.add_format({
    'font_size': font_size,
    'align': 'center'
})

worksheet.set_column('F:H', normal_column_width + 4, f_h_cell_format)

0

In [254]:
# Format for column I
i_cell_format = workbook.add_format({
    'font_size': font_size,
    'font_color': '#4F81BD',
    'num_format': '#,##0.00'
})

worksheet.set_column('I:I', normal_column_width, i_cell_format)

# worksheet.conditional_format('I6:I{}'.format(total_rows), {
#     'num_format': '#,##0.00'
# })

0

In [255]:
# Format for column K to M
k_m_cell_format = workbook.add_format({
    'font_size': font_size,
    'num_format': '#,##0.00'
})

worksheet.set_column('K:M', normal_column_width, k_m_cell_format)

0

In [256]:
writer.save()

In [20]:
# General column format
cell_format = workbook.add_format({
    'border':1
})

worksheet.set_column('A:M', 18, cell_format)
worksheet.set_column('E:E', 50, cell_format)

0

In [22]:
# Add number format
def set_number_columns_format(cell_format):
    number_format = workbook.add_format({
        'num_format': '#,##0.00'
    })
    worksheet.conditional_format('I4:M3411', {
        'type': 'cell',
        'criteria': '>',
        'value': 0,
        'format': number_format
    })
    
    number_cell_format = workbook.add_format({
        'font_color': '#4F81BD'
    })
    
    worksheet.set_column('I:I', 18, cell_format)
    

set_number_columns_format(number_cell_format)


In [25]:
# Style clean up
clean_format = workbook.add_format({
    'border': 0
})

# worksheet.set_column('A1:M3', None, clean_format)
for index in range(3):
    worksheet.set_row(index, None, clean_format)

In [26]:
worksheet.conditional_format('A1:M3411', {'type':   'blanks',
                                       'format': clean_format})